In [1]:
from astropy.io import ascii
from astropy.table import vstack, Table, unique, QTable
from astropy.coordinates import SkyCoord 
import astropy.units as u
from astropy import table, log
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord, Distance, Angle
from astropy.time import Time

import os
from IPython.display import display
from multiprocessing import set_start_method

from math import sqrt, cos

In [2]:
# google colab read file in: 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# after mounting drive, copy the path of the directory where the files are stored
save_path='/content/drive/MyDrive/NOFS copy/WiserLabBackup/QueryResults'

In [4]:
path = '/content/drive/MyDrive/NOFS copy/wdstab6-27.ecsv'
wdstab = Table.read(path, header_start=0, data_start=1) 

In [5]:
# # Read the file

# # vayu's lab comp:
# save_path = 'C:/Users/sc36/Documents/DaphneUSNO/NOFS copy-20230218T215456Z-001/NOFS copy/QueryResults'

# # wiser's lab comp:
# save_path = '/home/student/djz7128/djz_NOFS/QueryResults'

qrt ='{0}/stack_query_results_table.ecsv'.format(save_path) 
ie = '{0}/stack_index_error_queries.ecsv'.format(save_path)
query_results_table = QTable.read(qrt, header_start=0, data_start=1)
index_error_queries = Table.read(ie, header_start=0, data_start=1)


## Assign Units: 

## DEGREES 

deg_columns = ['ra_a', 'ra_b', 'dec_a', 'dec_b']
for deg_column in deg_columns:
    query_results_table[deg_column] = query_results_table[deg_column]*u.deg
    
## MAS

mas_columns = ['ra_error_a', 'ra_error_b', 'dec_error_a', 'dec_error_b', 'parallax_a', 'parallax_b', 'parallax_error_a', 'parallax_error_b']
for mas_column in mas_columns:
    query_results_table[mas_column] = query_results_table[mas_column]*u.mas

## MAS/YR
mas_p_year_columns = ['pmra_a', 'pmra_b', 'pmra_error_a', 'pmra_error_b', 'pmdec_a', 'pmdec_b', 'pmdec_error_a', 'pmdec_error_b']
for mas_p_year_column in mas_p_year_columns:
    query_results_table[mas_p_year_column] = query_results_table[mas_p_year_column]*u.mas/u.yr

### Find duplicate rows

In [6]:
duplicates = []
for row in range(len(query_results_table)):
  if row%5000==0:
    print(row)
  row_entry = query_results_table[row]
  for comprow in range(row+1, len(query_results_table)):
    if query_results_table[comprow]==row_entry:
      duplicates.append(comprow)
      continue
    else:
      break


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


In [9]:
query_results_table

wds_identifier,wds_rownum,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b
,,,,,,deg,deg,mas,mas,deg,deg,mas,mas,mas,mas,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str13,int64,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
00000+3852BU,3,2881742976228918912,2881742980523997824,2016.0,2016.0,0.007304252161929567,0.005058967400198452,0.016427384689450264,0.02496100589632988,38.85872352417506,38.859269115054886,0.014054271392524242,0.019697383046150208,2.955704163346745,2.992686610903894,0.027848364785313606,0.04275492951273918,106.1356430053711,69.99629211425781,5.04121006557981,5.761322176559112,0.01744064688682556,0.027374057099223137,-2.2337674557458724,-2.4055595285991993,0.015975115820765495,0.022997289896011353,-5.568790435791016,nan,0.7653659582138062,nan,31.0,31.0,20.0,18.0,0.028870442882180214,0.046229343861341476,1.2354885339736938,1.0191636085510254,11.427901268005371,6.596024990081787,1726.3349609375,2948.464111328125,11.674772262573242,6.584270477294922,90.04275512695312,1470.9251708984375,10.888983726501465,6.558365345001221,262.5050964355469,930.067626953125,0.7857885360717773,0.025905132293701172,1.2692034244537354,1.16890287399292
00001+7727LOC,6,540288988710861824,540288988710861952,2016.0,2016.0,0.025039271133275212,0.03472491134344492,0.050575174391269684,0.013834286481142044,77.44280871485321,77.44362726856326,0.04344682767987251,0.01188855990767479,6.7716161348462895,6.894360456576272,0.05322949215769768,0.014487730339169502,127.21549224853516,475.8758239746094,85.8194419212781,85.46432115647032,0.06538749486207962,0.017955169081687927,36.5657437504867,36.795972656600284,0.05974858999252319,0.016469508409500122,nan,-27.13381004333496,nan,8.205977439880371,31.0,31.0,25.0,25.0,0.09252406656742096,0.025319892913103104,1.21744704246521,1.072059154510498,16.57539176940918,14.10311508178711,904.2947387695312,1744.7960205078125,18.147706985473633,15.268089294433594,80.51824951171875,421.7792663574219,15.334451675415039,13.029826164245605,532.9720458984375,699.989501953125,2.8132553100585938,2.2382631301879883,1.4904694557189941,1.3791841268539429
00001+5400ES,7,396305497218596096,396305570238406144,2016.0,2016.0,0.030011439476742407,0.028112788664484196,0.008326414041221142,0.015584493987262249,53.999576973903956,54.00012486314701,0.008348978124558926,0.014539013616740704,2.384840253174304,2.3315141654599225,0.013162785209715366,0.023133113980293274,181.18052673339844,100.78687286376953,-0.9147508384598348,-1.064158531848239,0.010332660749554634,0.01920725405216217,-8.803328013644308,-8.914049677675674,0.011068823747336864,0.01896110363304615,-3.078555107116699,0.7382858991622925,1.3963912725448608,1.2604676485061646

In [8]:
# query_results_table.remove_rows(duplicates)

### Find rows that were mistakenly added (should be index error query objects)


In [10]:
wds_id_error_rows = []
wrong_ids = []

for row in range(len(query_results_table)):
    if row%5000==0:
        print(row)
    ra_a, ra_b = query_results_table[row]['ra_a'], query_results_table[row]['ra_b']
    dec_a, dec_b = query_results_table[row]['dec_a'], query_results_table[row]['dec_b']
    id1 = query_results_table['wds_identifier'][row]
    for comprow in range(row+1, len(query_results_table)):
        compid = query_results_table['wds_identifier'][comprow]
        if compid != id1:

            nextid = query_results_table['wds_identifier'][comprow]
            # for nextrow in range(comprow, 3):
            # if query_results_table[nextrow]['wds_identifier'] == nextid:
            #   new_ra_a, new_ra_b = query_results_table[nextrow]['ra_a'], query_results_table[nextrow]['ra_b']
            #   new_dec_a, new_dec_b = query_results_table[nextrow]['dec_a'], query_results_table[nextrow]['dec_b']
            #   if ra_a == new_ra_a and ra_b == new_ra_b and dec_a == new_dec_a and dec_b == new_dec_b:
            #     wds_id_error_rows.append(nextrow)
            #     print(compid)

            if query_results_table[comprow]['wds_identifier'] == nextid:
                new_ra_a, new_ra_b = query_results_table[comprow]['ra_a'], query_results_table[comprow]['ra_b']
                new_dec_a, new_dec_b = query_results_table[comprow]['dec_a'], query_results_table[comprow]['dec_b']

            if ra_a == new_ra_a and ra_b == new_ra_b and dec_a == new_dec_a and dec_b == new_dec_b:
                wds_id_error_rows.append(comprow)
                wrong_ids.append(nextid)
                # print(compid)
                break
            else:
                break

        
        


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000


### Test the rows to make sure they are actually supposed to be index error query objects

In [ ]:
wds_id_error_rows


In [12]:
wdstab.add_index('WDS Identifier')
id= '23592+4112HJL'
id= '23563+6128STI'
id='00082+0937LDS'
wdsrow=wdstab.loc_indices[id]
wdsrow

754

##### check how many objects are far away from their WDS coord

In [ ]:
# wdstab.add_index('WDS Identifier')

# mismatch_rows =  []
# for row in range(len(query_results_table)):
#   if row%5000==0:
#     print(row)
#   try:
#     rra_a = query_results_table[row]['ra_a']/u.deg
#     rdec_a = query_results_table[row]['dec_a']/u.deg
#     id = query_results_table[row]['wds_identifier']
#     wdsrow = wdstab.loc_indices[id]
#     wdsra_a = wdstab[wdsrow]['RApri-prepped']
#     wdsdec_a = wdstab[wdsrow]['DECpri-prepped']
#     wdsra_b = wdstab[wdsrow]['RAsec-prepped']
#     wdsdec_b = wdstab[wdsrow]['DECsec-prepped']


#     if abs(rra-wdsra_a)>2:
#       if abs(rra-wdsra_b)>2:
#         mismatch_rows.append(row)
#     elif abs(rdec-wdsdec_a)>2:
#       if abs(rdec-wdsdec_b)>2:
#         mismatch_rows.append(row)
#   except:
#     pass

### Put mistaken objects in the correct file

In [15]:
# for row in wds_id_error_rows:
#   index_error_queries.add_row()
#   index_error_queries[-1]['wds_identifier'] = query_results_table[row]['wds_identifier']
#   index_error_queries[-1]['wds_rownum'] = query_results_table[row]['wds_rownum']


In [18]:
# query_results_table.remove_rows(wds_id_error_rows)

In [37]:
len(wds_id_error_rows)

14891

## Remove Empty Rows

In [3]:
zeros = []
for row in range(len(query_results_table)):
#   if row%5000==0:
#     print(row)

    if query_results_table['ra_a'][row] == 0:
      zeros.append(row)

query_results_table.remove_rows(zeros)

NameError: name 'query_results_table' is not defined

In [20]:

ascii.write(query_results_table, '{path}/stack_query_results_table_fixed4-20.ecsv'.format(path = save_path), format='ecsv',overwrite=True)
ascii.write(query_results_table, '{path}/stack_query_results_table_fixed4-20.csv'.format(path = save_path), format='csv',overwrite=True)
    
ascii.write(index_error_queries, '{path}/stack_index_error_queries_fixed4-20.ecsv'.format(path = save_path), format='ecsv',overwrite=True)
ascii.write(index_error_queries, '{path}/stack_index_error_queries_fixed4-20.csv'.format(path = save_path), format='csv',overwrite=True)

In [21]:
query_results_table

wds_identifier,wds_rownum,source_id_a,source_id_b,ref_epoch_a,ref_epoch_b,ra_a,ra_b,ra_error_a,ra_error_b,dec_a,dec_b,dec_error_a,dec_error_b,parallax_a,parallax_b,parallax_error_a,parallax_error_b,parallax_over_error_a,parallax_over_error_b,pmra_a,pmra_b,pmra_error_a,pmra_error_b,pmdec_a,pmdec_b,pmdec_error_a,pmdec_error_b,radial_velocity_a,radial_velocity_b,radial_velocity_error_a,radial_velocity_error_b,astrometric_params_solved_a,astrometric_params_solved_b,visibility_periods_used_a,visibility_periods_used_b,astrometric_sigma5d_max_a,astrometric_sigma5d_max_b,ruwe_a,ruwe_b,phot_g_mean_mag_a,phot_g_mean_mag_b,phot_g_mean_flux_over_error_a,phot_g_mean_flux_over_error_b,phot_bp_mean_mag_a,phot_bp_mean_mag_b,phot_bp_mean_flux_over_error_a,phot_bp_mean_flux_over_error_b,phot_rp_mean_mag_a,phot_rp_mean_mag_b,phot_rp_mean_flux_over_error_a,phot_rp_mean_flux_over_error_b,bp_rp_a,bp_rp_b,phot_bp_rp_excess_factor_a,phot_bp_rp_excess_factor_b
,,,,,,deg,deg,mas,mas,deg,deg,mas,mas,mas,mas,mas,mas,,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,,,,,,,,,,,,
str13,int64,str19,str19,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
00000+3852BU,3,2881742976228918912,2881742980523997824,2016.0,2016.0,0.007304252161929567,0.005058967400198452,0.016427384689450264,0.02496100589632988,38.85872352417506,38.859269115054886,0.014054271392524242,0.019697383046150208,2.955704163346745,2.992686610903894,0.027848364785313606,0.04275492951273918,106.1356430053711,69.99629211425781,5.04121006557981,5.761322176559112,0.01744064688682556,0.027374057099223137,-2.2337674557458724,-2.4055595285991993,0.015975115820765495,0.022997289896011353,-5.568790435791016,nan,0.7653659582138062,nan,31.0,31.0,20.0,18.0,0.028870442882180214,0.046229343861341476,1.2354885339736938,1.0191636085510254,11.427901268005371,6.596024990081787,1726.3349609375,2948.464111328125,11.674772262573242,6.584270477294922,90.04275512695312,1470.9251708984375,10.888983726501465,6.558365345001221,262.5050964355469,930.067626953125,0.7857885360717773,0.025905132293701172,1.2692034244537354,1.16890287399292
00001+7727LOC,6,540288988710861824,540288988710861952,2016.0,2016.0,0.025039271133275212,0.03472491134344492,0.050575174391269684,0.013834286481142044,77.44280871485321,77.44362726856326,0.04344682767987251,0.01188855990767479,6.7716161348462895,6.894360456576272,0.05322949215769768,0.014487730339169502,127.21549224853516,475.8758239746094,85.8194419212781,85.46432115647032,0.06538749486207962,0.017955169081687927,36.5657437504867,36.795972656600284,0.05974858999252319,0.016469508409500122,nan,-27.13381004333496,nan,8.205977439880371,31.0,31.0,25.0,25.0,0.09252406656742096,0.025319892913103104,1.21744704246521,1.072059154510498,16.57539176940918,14.10311508178711,904.2947387695312,1744.7960205078125,18.147706985473633,15.268089294433594,80.51824951171875,421.7792663574219,15.334451675415039,13.029826164245605,532.9720458984375,699.989501953125,2.8132553100585938,2.2382631301879883,1.4904694557189941,1.3791841268539429
00001+5400ES,7,396305497218596096,396305570238406144,2016.0,2016.0,0.030011439476742407,0.028112788664484196,0.008326414041221142,0.015584493987262249,53.999576973903956,54.00012486314701,0.008348978124558926,0.014539013616740704,2.384840253174304,2.3315141654599225,0.013162785209715366,0.023133113980293274,181.18052673339844,100.78687286376953,-0.9147508384598348,-1.064158531848239,0.010332660749554634,0.01920725405216217,-8.803328013644308,-8.914049677675674,0.011068823747336864,0.01896110363304615,-3.078555107116699,0.7382858991622925,1.3963912725448608,1.2604676485061646

In [22]:
index_error_queries

wds_identifier,wds_rownum
str13,float64
00000+7530A,0.0
00000+4004ES,1.0
00000+0044SKF,4.0
00000-0530OCC,5.0
00001+0638SLW,10.0
00001+0022SKF,11.0
00002+4119TDS,15.0
00002+4119FYM,16.0
00002+3613TDS,17.0
